# Get track 

In [1]:
import duckdb
import pandas as pd
import glob 
import os.path

from shapely.geometry import Point

## Import data and quick inspection

Data is in the parent dir so get all files, also used pyspark to format the data

In [2]:
# Get list of parquet files to import into duckdb
data_dir = '../data/**/*.parquet'
data_list = list(filter(lambda x: not os.path.isdir(x), glob.glob(data_dir, recursive=True) ))

Load the data and GEOSPATIAL package

In [3]:
start_date = '2022-01-01 00:00:00'
stop_date = '2022-01-02 00:00:00'

In [4]:
conn = duckdb.connect()
conn.execute("INSTALL spatial; LOAD spatial;")
#conn.execute(f"CREATE VIEW ais AS SELECT * FROM read_parquet({data_list})")
conn.execute(f"""
    CREATE VIEW ais AS SELECT * FROM read_parquet({data_list})
        WHERE basedatetime BETWEEN '{start_date}' AND '{stop_date}'
""")

Print out a table view

In [5]:
data = conn.execute("PRAGMA table_info(ais)").fetchall()

# Convert DuckDB query results into a pandas DataFrame
columns = [str(row[1]) for row in data]
top_rows = pd.DataFrame(conn.execute("SELECT * FROM ais LIMIT 10").fetchall(), columns=columns)

# Print the first few rows of the resulting DataFrame
print(top_rows)

        MMSI        BaseDateTime        LAT         LON  SOG         COG  \
0  368084090 2022-01-01 05:00:00  29.931740  -89.992432  6.0  296.200012   
1  368140160 2022-01-01 05:00:00  30.334749  -87.144287  0.0  312.000000   
2  366941830 2022-01-01 05:00:02  29.309191  -94.797020  0.0  180.199997   
3  316005971 2022-01-01 05:00:06  46.502682  -84.356743  2.4  258.600006   
4  316004054 2022-01-01 05:00:07  46.503262  -84.375061  0.3   61.900002   
5  366863870 2022-01-01 05:00:08  41.087479  -72.358139  0.0  215.300003   
6  368118150 2022-01-01 05:00:25  47.397888 -122.329910  0.0  360.000000   
7  338379253 2022-01-01 05:00:36  33.619968 -117.895912  0.0  360.000000   
8  368195460 2022-01-01 05:00:41  37.643299 -122.329872  4.7  332.500000   
9  367614790 2022-01-01 05:00:47  21.316730 -157.878204  0.0  203.699997   

   Heading       VesselName         IMO CallSign  VesselType  Status  Length  \
0    299.0  LARRY B WHIPPLE        None  WDK7401          57    12.0    23.0   
1  

Get a single vessel and see if we can make a track

In [6]:
#track_data = conn.execute("SELECT mmsi,lat,lon,basedatetime FROM ais WHERE mmsi ='367615680'").fetchall()
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS sample_track (
        mmsi INT,
        lat FLOAT,
        lon FLOAT,
        basedatetime DATETIME
    )
""")

cur.execute("""
    INSERT INTO sample_track (mmsi, lat, lon, basedatetime)
    SELECT mmsi, lat, lon, basedatetime
    FROM ais
    WHERE mmsi = '367615680'
    AND 
    basedatetime BETWEEN '2022-01-01 00:00:00' AND '2022-01-02 00:00:00'
    ORDER BY basedatetime ASC
""")
conn.commit()

In [7]:
# Fetch data from the new table into a pandas DataFrame
track_data = pd.DataFrame(cur.execute("SELECT * FROM sample_track").fetchall(), columns=['mmsi', 'lat', 'lon', 'basedatetime'])
print(track_data.head())

        mmsi        lat        lon        basedatetime
0  367615680  28.383039 -96.540581 2022-01-01 05:01:35
1  367615680  28.384060 -96.538338 2022-01-01 05:03:04
2  367615680  28.384760 -96.536583 2022-01-01 05:04:15
3  367615680  28.386580 -96.532654 2022-01-01 05:06:55
4  367615680  28.387690 -96.530212 2022-01-01 05:08:34


## Plot Track

In [9]:
import folium

In [10]:
cur = conn.cursor()
points_list = cur.execute("SELECT lat, lon FROM sample_track").fetchall()
conn.commit()

In [12]:
zoom_point = points_list[len(points_list) // 2]
m = folium.Map(location=zoom_point, zoom_start=12)
for p in points_list:
    folium.Marker([p[0], p[1]]).add_to(m)
m.save('map.html')